# 12. Рекуррентные нейронные сети и LSTM для временных рядов

## Введение

Рекуррентные нейронные сети (RNN) и их улучшенные варианты, такие как LSTM (Long Short-Term Memory) и GRU (Gated Recurrent Unit), представляют собой мощный инструмент для моделирования временных рядов. В отличие от классических статистических методов, глубокое обучение способно автоматически извлекать сложные нелинейные зависимости из данных.

## 1. Теоретические основы

### 1.1 Рекуррентные нейронные сети (RNN)

RNN - это класс нейронных сетей, которые имеют циклические связи, позволяющие информации сохраняться между шагами обработки последовательности.

**Основное уравнение RNN:**

$$h_t = \tanh(W_{hh}h_{t-1} + W_{xh}x_t + b_h)$$

$$y_t = W_{hy}h_t + b_y$$

где:
- $h_t$ - скрытое состояние в момент времени $t$
- $x_t$ - входной вектор в момент времени $t$
- $y_t$ - выходной вектор в момент времени $t$
- $W$ - матрицы весов
- $b$ - векторы смещений

**Проблемы простых RNN:**
- Проблема затухающего градиента (vanishing gradient)
- Проблема взрывающегося градиента (exploding gradient)
- Сложность в обучении долгосрочных зависимостей

### 1.2 LSTM (Long Short-Term Memory)

LSTM были разработаны для решения проблемы долгосрочных зависимостей. Они используют специальную архитектуру с "воротами" (gates), которые регулируют поток информации.

**Основные компоненты LSTM:**

1. **Forget Gate (Вентиль забывания):** решает, какую информацию удалить из состояния ячейки
   $$f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$$

2. **Input Gate (Входной вентиль):** решает, какую новую информацию добавить
   $$i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$$
   $$\tilde{C}_t = \tanh(W_C \cdot [h_{t-1}, x_t] + b_C)$$

3. **Cell State Update (Обновление состояния ячейки):**
   $$C_t = f_t \ast C_{t-1} + i_t \ast \tilde{C}_t$$

4. **Output Gate (Выходной вентиль):**
   $$o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$$
   $$h_t = o_t \ast \tanh(C_t)$$

где $\sigma$ - сигмоидная функция, $\ast$ - поэлементное умножение.

### 1.3 GRU (Gated Recurrent Unit)

GRU - упрощенная версия LSTM с меньшим числом параметров:

$$z_t = \sigma(W_z \cdot [h_{t-1}, x_t])$$ (update gate)

$$r_t = \sigma(W_r \cdot [h_{t-1}, x_t])$$ (reset gate)

$$\tilde{h}_t = \tanh(W \cdot [r_t \ast h_{t-1}, x_t])$$

$$h_t = (1 - z_t) \ast h_{t-1} + z_t \ast \tilde{h}_t$$

## 2. Практическая реализация

### 2.1 Подготовка данных

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Для работы с LSTM понадобится TensorFlow/Keras или PyTorch
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Генерация синтетических данных временного ряда
np.random.seed(42)
time = np.arange(0, 1000)
trend = 0.02 * time
seasonality = 10 * np.sin(2 * np.pi * time / 50)
noise = np.random.randn(len(time)) * 2
ts_data = trend + seasonality + noise + 50

# Создание DataFrame
df = pd.DataFrame({'time': time, 'value': ts_data})

# Визуализация
plt.figure(figsize=(14, 5))
plt.plot(df['time'], df['value'], label='Временной ряд')
plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Исходный временной ряд')
plt.legend()
plt.show()

print(f"Размер данных: {len(df)}")
print(df.head())

### 2.2 Подготовка данных для LSTM

Для обучения LSTM необходимо преобразовать временной ряд в формат (samples, timesteps, features). Создадим функцию для создания последовательностей с окном (window).

In [ ]:
def create_sequences(data, window_size, horizon=1):
    """
    Создание последовательностей для обучения LSTM
    
    Parameters:
    -----------
    data : array-like
        Исходный временной ряд
    window_size : int
        Размер окна (количество предыдущих наблюдений)
    horizon : int
        Горизонт прогнозирования (количество шагов вперед)
    
    Returns:
    --------
    X, y : tuple of arrays
        Входные последовательности и целевые значения
    """
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size:i + window_size + horizon])
    return np.array(X), np.array(y)

# Нормализация данных (важно для нейронных сетей)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['value']])

# Параметры
window_size = 50  # Используем 50 предыдущих наблюдений
horizon = 1       # Прогнозируем 1 шаг вперед
train_size = int(len(scaled_data) * 0.8)

# Разделение на обучающую и тестовую выборки
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - window_size:]  # включаем окно для первого предсказания

# Создание последовательностей
X_train, y_train = create_sequences(train_data, window_size, horizon)
X_test, y_test = create_sequences(test_data, window_size, horizon)

# Reshape для LSTM: (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

### 2.3 Построение модели LSTM

In [ ]:
# Создание архитектуры LSTM модели
def build_lstm_model(input_shape, units=[50, 50], dropout_rate=0.2):
    """
    Построение LSTM модели
    
    Parameters:
    -----------
    input_shape : tuple
        Размерность входа (timesteps, features)
    units : list
        Список размерностей LSTM слоев
    dropout_rate : float
        Коэффициент dropout для регуляризации
    """
    model = Sequential()
    
    # Первый LSTM слой
    model.add(LSTM(units=units[0], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Второй LSTM слой
    model.add(LSTM(units=units[1], return_sequences=False))
    model.add(Dropout(dropout_rate))
    
    # Выходной слой
    model.add(Dense(units=1))
    
    # Компиляция
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    
    return model

# Построение модели
input_shape = (X_train.shape[1], X_train.shape[2])
lstm_model = build_lstm_model(input_shape, units=[64, 32], dropout_rate=0.2)

# Вывод архитектуры
lstm_model.summary()

### 2.4 Обучение модели

In [ ]:
# Настройка колбэков
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Обучение модели
history = lstm_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Визуализация процесса обучения
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Ошибка обучения и валидации')
axes[0].legend()
axes[0].grid(True)

# MAE
axes[1].plot(history.history['mae'], label='Training MAE')
axes[1].plot(history.history['val_mae'], label='Validation MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].set_title('Средняя абсолютная ошибка')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

### 2.5 Прогнозирование и оценка

In [ ]:
# Получение предсказаний
train_predictions = lstm_model.predict(X_train)
test_predictions = lstm_model.predict(X_test)

# Обратная трансформация (денормализация)
train_predictions = scaler.inverse_transform(train_predictions)
y_train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
test_predictions = scaler.inverse_transform(test_predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Вычисление метрик
train_rmse = np.sqrt(mean_squared_error(y_train_actual, train_predictions))
train_mae = mean_absolute_error(y_train_actual, train_predictions)
test_rmse = np.sqrt(mean_squared_error(y_test_actual, test_predictions))
test_mae = mean_absolute_error(y_test_actual, test_predictions)

print("=" * 50)
print("Метрики качества модели LSTM")
print("=" * 50)
print(f"Train RMSE: {train_rmse:.4f}")
print(f"Train MAE: {train_mae:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print("=" * 50)

In [ ]:
# Визуализация предсказаний
plt.figure(figsize=(16, 6))

# Индексы для построения
train_indices = range(window_size, window_size + len(train_predictions))
test_indices = range(train_size, train_size + len(test_predictions))

# Исходные данные
plt.plot(df['time'], df['value'], label='Исходные данные', alpha=0.6, linewidth=1)

# Предсказания на обучающей выборке
plt.plot(train_indices, train_predictions, label='Предсказания (train)', alpha=0.8, linewidth=2)

# Предсказания на тестовой выборке
plt.plot(test_indices, test_predictions, label='Предсказания (test)', alpha=0.8, linewidth=2, color='red')

# Вертикальная линия разделения
plt.axvline(x=train_size, color='black', linestyle='--', label='Train/Test split')

plt.xlabel('Время')
plt.ylabel('Значение')
plt.title('Прогнозирование временного ряда с помощью LSTM')
plt.legend()
plt.grid(True)
plt.show()

### 2.6 Сравнение с GRU

In [ ]:
# Построение GRU модели для сравнения
def build_gru_model(input_shape, units=[50, 50], dropout_rate=0.2):
    model = Sequential()
    model.add(GRU(units=units[0], return_sequences=True, input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(GRU(units=units[1], return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# Обучение GRU
gru_model = build_gru_model(input_shape, units=[64, 32], dropout_rate=0.2)
gru_history = gru_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

# Предсказания GRU
gru_test_predictions = gru_model.predict(X_test)
gru_test_predictions = scaler.inverse_transform(gru_test_predictions)

# Метрики GRU
gru_test_rmse = np.sqrt(mean_squared_error(y_test_actual, gru_test_predictions))
gru_test_mae = mean_absolute_error(y_test_actual, gru_test_predictions)

print("\nСравнение LSTM и GRU:")
print(f"LSTM Test RMSE: {test_rmse:.4f} | GRU Test RMSE: {gru_test_rmse:.4f}")
print(f"LSTM Test MAE: {test_mae:.4f} | GRU Test MAE: {gru_test_mae:.4f}")

### 2.7 Многошаговое прогнозирование

In [ ]:
def multi_step_forecast(model, initial_sequence, scaler, n_steps):
    """
    Многошаговое прогнозирование
    
    Parameters:
    -----------
    model : keras.Model
        Обученная модель
    initial_sequence : array
        Начальная последовательность
    scaler : MinMaxScaler
        Скейлер для денормализации
    n_steps : int
        Количество шагов прогноза
    """
    forecast = []
    current_sequence = initial_sequence.copy()
    
    for _ in range(n_steps):
        # Предсказание следующего значения
        pred = model.predict(current_sequence.reshape(1, window_size, 1), verbose=0)
        forecast.append(pred[0, 0])
        
        # Обновление последовательности (скользящее окно)
        current_sequence = np.append(current_sequence[1:], pred[0, 0])
    
    # Денормализация
    forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))
    return forecast

# Многошаговый прогноз на 100 шагов вперед
n_forecast_steps = 100
last_sequence = scaled_data[train_size - window_size:train_size].flatten()
multi_step_pred = multi_step_forecast(lstm_model, last_sequence, scaler, n_forecast_steps)

# Визуализация
plt.figure(figsize=(14, 6))
plt.plot(df['time'][:train_size], df['value'][:train_size], label='Обучающая выборка')
plt.plot(df['time'][train_size:train_size + n_forecast_steps], 
         df['value'][train_size:train_size + n_forecast_steps], 
         label='Фактические значения', alpha=0.7)
plt.plot(range(train_size, train_size + n_forecast_steps), 
         multi_step_pred, 
         label='Многошаговый прогноз', linestyle='--', linewidth=2)
plt.axvline(x=train_size, color='black', linestyle='--', alpha=0.5)
plt.xlabel('Время')
plt.ylabel('Значение')
plt.title(f'Многошаговое прогнозирование на {n_forecast_steps} шагов вперед')
plt.legend()
plt.grid(True)
plt.show()

## 3. Продвинутые техники

### 3.1 Bidirectional LSTM

Двунаправленные LSTM обрабатывают последовательность в обоих направлениях, что может улучшить качество прогнозов.

In [ ]:
from tensorflow.keras.layers import Bidirectional

# Bidirectional LSTM модель
bi_lstm_model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape),
    Dropout(0.2),
    Bidirectional(LSTM(32)),
    Dropout(0.2),
    Dense(1)
])

bi_lstm_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
bi_lstm_model.summary()

### 3.2 Encoder-Decoder архитектура для sequence-to-sequence

Для многошагового прогнозирования более эффективна архитектура encoder-decoder.

In [ ]:
from tensorflow.keras.layers import RepeatVector, TimeDistributed

# Параметры для seq2seq
n_features = 1
forecast_horizon = 10  # Прогнозируем 10 шагов вперед

# Подготовка данных для seq2seq
X_seq2seq_train, y_seq2seq_train = create_sequences(train_data, window_size, forecast_horizon)
X_seq2seq_test, y_seq2seq_test = create_sequences(test_data, window_size, forecast_horizon)

X_seq2seq_train = X_seq2seq_train.reshape((X_seq2seq_train.shape[0], window_size, n_features))
X_seq2seq_test = X_seq2seq_test.reshape((X_seq2seq_test.shape[0], window_size, n_features))

# Encoder-Decoder модель
encoder_decoder_model = Sequential([
    # Encoder
    LSTM(64, activation='relu', input_shape=(window_size, n_features)),
    RepeatVector(forecast_horizon),
    
    # Decoder
    LSTM(64, activation='relu', return_sequences=True),
    TimeDistributed(Dense(1))
])

encoder_decoder_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
encoder_decoder_model.summary()

print(f"\nФорма входных данных: {X_seq2seq_train.shape}")
print(f"Форма целевых данных: {y_seq2seq_train.shape}")

### 3.3 Attention механизм

Механизм внимания позволяет модели фокусироваться на наиболее релевантных частях входной последовательности.

In [ ]:
from tensorflow.keras.layers import Layer, Multiply, Permute, Lambda
import tensorflow.keras.backend as K

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', 
                                 shape=(input_shape[-1], 1),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(name='attention_bias',
                                 shape=(input_shape[1], 1),
                                 initializer='zeros',
                                 trainable=True)
        super(AttentionLayer, self).build(input_shape)
    
    def call(self, x):
        # Вычисление весов внимания
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# LSTM модель с Attention
lstm_attention_model = Sequential([
    LSTM(64, return_sequences=True, input_shape=input_shape),
    Dropout(0.2),
    LSTM(32, return_sequences=True),
    Dropout(0.2),
    AttentionLayer(),
    Dense(1)
])

lstm_attention_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("LSTM модель с механизмом Attention создана")

## 4. Практические рекомендации

### 4.1 Выбор гиперпараметров

**Размер окна (window_size):**
- Для данных с сезонностью: минимум один полный сезон
- Для данных с трендом: 50-200 наблюдений
- Экспериментируйте с разными значениями

**Количество LSTM слоев и нейронов:**
- Начните с 1-2 слоев по 32-64 нейрона
- Увеличивайте при наличии сложных паттернов
- Следите за переобучением

**Dropout:**
- Типичные значения: 0.2-0.4
- Увеличивайте при переобучении

**Batch size:**
- Меньшие значения (16-32): более шумное обучение, но может выйти из локальных минимумов
- Большие значения (64-128): более стабильное обучение

### 4.2 Когда использовать LSTM vs классические методы

**LSTM предпочтительны когда:**
- Большой объем данных (>1000 наблюдений)
- Нелинейные зависимости
- Множественные переменные (многомерные временные ряды)
- Долгосрочные зависимости

**Классические методы (ARIMA, Exponential Smoothing) предпочтительны когда:**
- Малый объем данных (<500 наблюдений)
- Необходима интерпретируемость
- Простые линейные паттерны
- Ограниченные вычислительные ресурсы

### 4.3 Предотвращение переобучения

1. **Dropout** - регуляризация
2. **Early Stopping** - остановка при ухудшении валидационной ошибки
3. **L1/L2 регуляризация** весов
4. **Уменьшение сложности модели**
5. **Аугментация данных** (добавление шума, сдвиги)
6. **Cross-validation** для временных рядов

## 5. Задания для самостоятельной работы

### Задание 1: Базовое (обязательное)

Загрузите данные о пассажиропотоке авиакомпаний (Air Passengers dataset) и:
1. Подготовьте данные для LSTM с window_size = 12
2. Постройте и обучите LSTM модель
3. Сравните качество с ARIMA моделью из предыдущих ноутбуков
4. Визуализируйте результаты

**Критерии оценки:**
- Правильная подготовка данных
- Работающая модель LSTM
- Корректные метрики качества
- Визуализация предсказаний

In [ ]:
# Место для решения задания 1
# TODO: Ваш код здесь

### Задание 2: Продвинутое

Реализуйте эксперимент по подбору гиперпараметров:
1. Создайте функцию для обучения LSTM с различными параметрами
2. Переберите различные комбинации:
   - window_size: [20, 50, 100]
   - units: [[32, 16], [64, 32], [128, 64]]
   - dropout_rate: [0.1, 0.2, 0.3]
3. Сохраните результаты в DataFrame
4. Постройте визуализацию зависимости качества от гиперпараметров
5. Выберите лучшую конфигурацию

**Дополнительно:** Используйте Keras Tuner или Optuna для автоматического подбора

In [ ]:
# Место для решения задания 2
# TODO: Ваш код здесь

### Задание 3: Исследовательское

Реализуйте многомерный прогноз временного ряда:
1. Найдите или создайте датасет с несколькими связанными временными рядами (например, температура, влажность, давление)
2. Постройте многомерную LSTM модель (multivariate forecasting)
3. Сравните качество прогноза при использовании:
   - Только целевой переменной (univariate)
   - Всех переменных (multivariate)
4. Проанализируйте, какие переменные наиболее важны
5. Реализуйте механизм Attention и оцените его влияние

**Критерии оценки:**
- Корректная работа с многомерными данными
- Сравнительный анализ
- Интерпретация результатов
- Реализация Attention

In [ ]:
# Место для решения задания 3
# TODO: Ваш код здесь

## Контрольные вопросы

1. В чем заключается проблема затухающего градиента в простых RNN?
2. Объясните назначение каждого из трех вентилей (gates) в LSTM.
3. Чем GRU отличается от LSTM? Когда стоит использовать GRU вместо LSTM?
4. Почему важна нормализация данных для нейронных сетей?
5. Как работает механизм Attention в контексте временных рядов?
6. Когда следует использовать Bidirectional LSTM?
7. В чем разница между one-step и multi-step forecasting?
8. Как предотвратить переобучение в LSTM моделях?
9. Почему encoder-decoder архитектура эффективна для sequence-to-sequence задач?
10. В каких ситуациях LSTM модели превосходят классические статистические методы?

## Дополнительные материалы

**Статьи:**
- Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural computation, 9(8), 1735-1780.
- Cho, K., et al. (2014). Learning phrase representations using RNN encoder-decoder for statistical machine translation.
- Bahdanau, D., Cho, K., & Bengio, Y. (2014). Neural machine translation by jointly learning to align and translate.

**Онлайн ресурсы:**
- [Colah's Blog: Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
- [Keras Time Series Forecasting Tutorial](https://keras.io/examples/timeseries/)
- [TensorFlow Time Series Tutorial](https://www.tensorflow.org/tutorials/structured_data/time_series)

**Книги:**
- Goodfellow, I., Bengio, Y., & Courville, A. (2016). Deep Learning. MIT Press.
- Chollet, F. (2021). Deep Learning with Python, Second Edition. Manning Publications.